In [80]:
import pandas as pd
from tqdm import tqdm
import re

In [43]:
bkms = pd.read_csv('bkms/Reactions_BKMS_2023-1.csv', sep = '\t')
#maybe include cofs?
metacyc = pd.read_csv('metacyc/21Nov2023_metacyc_reaction_smiles_no_cofs_with_sequences.tsv', sep = '\t')

/var/folders/cb/fq3bwnj51bv2d7d694kx77pm0000gp/T/ipykernel_31750/3767433003.py:1: DtypeWarning: Columns (5,7,9,10,16) have mixed types. Specify dtype option on import or set low_memory=False.
  bkms = pd.read_csv('bkms/Reactions_BKMS_2023-1.csv', sep = '\t')


In [108]:
bkms

,ID,EC_Number,Recommended_Name,Reaction,Reaction_ID_BRENDA,Reaction_ID_KEGG,Reaction_ID_MetaCyc,Reaction_ID_SABIO_RK,BRENDA_Pathway_Name,KEGG_Pathway_ID,KEGG_Pathway_Name,MetaCyc_Pathway_ID,MetaCyc_Pathway_Name,Stoichiometry_Check,Missing_Substrate,Missing_Product,Commentary_KEGG,Commentary_MetaCyc,Remark
0,1,5.3.3.3,vinylacetyl-CoA DELTA-isomerase,vinylacetyl-CoA = (E)-but-2-enoyl-CoA,BR1,R03031,VINYLACETYL-COA-DELTA-ISOMERASE-RXN,NaN,NaN,rn00650,Butanoate metabolism,NaN,NaN,NaN,NaN,NaN,NaN,NaN,natural substrates
1,2,6.3.2.1,pantoate-beta-alanine ligase (AMP-forming),ATP + (R)-pantoate + beta-alanine = AMP + diph...,BR101,R02473,PANTOATE-BETA-ALANINE-LIG-RXN,2406,pantothenate biosynthesis,rn00410; rn00770; rn01110,beta-Alanine metabolism; Pantothenate and CoA ...,PANTO-PWY,phosphopantothenate biosynthesis I,NaN,NaN,NaN,NaN,NaN,natural substrates
2,3,6.3.2.10,UDP-N-acetylmuramoyl-tripeptide-D-alanyl-D-ala...,ATP + UDP-N-acetylmuramoyl-L-alanyl-gamma-D-gl...,BR102,R04573,6.3.2.10-RXN,4244,peptidoglycan biosynthesis,rn00550,Peptidoglycan biosynthesis,PWY-6386,UDP-N-acetylmuramoyl-pentapeptide biosynthesis...,NaN,NaN,NaN,NaN,NaN,natural substrates
3,4,6.3.2.4,D-Alanine-D-alanine ligase,ATP + UDP-N-acetylmuramoyl-L-alanyl-gamma-D-gl...,BR102,R04573,6.3.2.10-RXN,4244,peptidoglycan biosynthesis,rn00550,Peptidoglycan biosynthesis,PWY-6386,UDP-N-acetylmuramoyl-pentapeptide biosynthesis...,NaN,NaN,NaN,NaN,NaN,natural substrates
4,5,6.3.2.11,carnosine synthase,ATP + L-histidine + beta-alanine = ADP + phosp...,BR103,R01164,CARNOSINE-SYNTHASE-RXN,NaN,NaN,rn00340; rn00410,Histidine metabolism; beta-Alanine metabolism,PWY66-420,carnosine biosynthesis,NaN,NaN,NaN,NaN,NaN,natural substrates
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42534,100104,3.1.13.4,poly(A)-specific ribonuclease,poly(A)-mRNA + H2O = 5'-AMP,BS92998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,generic compounds,NaN,NaN,NaN,NaN,"natural substrates, generic, nucleotide"
42535,100105,3.6.1.53,Mn2+-dependent ADP-ribose/CDP-alcohol diphosph...,"diadenosine 5',5''-P1, P2-diphosphate + H2O = ...",BS165445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,natural substrates
42536,100106,3.6.1.9,nucleotide diphosphatase,"diadenosine 5',5''-P1, P2-diphosphate + H2O = ...",BS165445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,natural substrates
42537,100108,3.1.4.17,"3',5'-cyclic-nucleotide phosphodiesterase","2',3'-cAMP + H2O = adenosine monophosphate",BS165471,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,natural substrates


In [45]:
def get_reaction_smiles_for_metacyc(reaction_id, metacyc_df = metacyc):
    match = metacyc_df[metacyc_df['UNIQUE-ID'] == reaction_id]['reaction_smiles'].values
    if len(match):
        return match[0]
  

In [46]:
for idx in bkms.index:
  # check if metacyc id exists for entry
  if not pd.isnull(bkms.loc[idx, 'Reaction_ID_MetaCyc']):
    reaction_smiles = get_reaction_smiles_for_metacyc(bkms.loc[idx, 'Reaction_ID_MetaCyc'])
    
  # else check if kegg id exists for entry
  elif not pd.isnull(bkms.loc[idx, 'Reaction_ID_KEGG']):
    pass
  # else check if brenda id exists for entry
  elif not pd.isnull(bkms.loc[idx, 'Reaction_ID_BRENDA']):
    pass
  # else check if sabio id exists for entry
  elif not pd.isnull(bkms.loc[idx, 'Reaction_ID_SABIO_RK'].isna):
    pass
  # else print index with warning message
  else:
    print(f'No reaction id found for index {idx}')


AttributeError: 'str' object has no attribute 'isna'

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
all_cpds_page = requests.get('https://www.genome.jp/dbget-bin/www_bfind_sub?dbkey=compound&keywords=C&mode=bfind&max_hit=nolimit')
soup = BeautifulSoup(all_cpds_page.text, 'html.parser')
a = soup.find_all('a')
cpd_links = [x.get('href') for x in a if x.get('href')]
mol_texts = [requests.get('https://www.genome.jp/entry/-f+m+'+c.split('/')[-1]).text for c in tqdm(cpd_links)]

  0%|          | 22/19272 [00:21<5:16:21,  1.01it/s]

KeyboardInterrupt: 

In [ ]:
from rdkit import Chem

In [ ]:
Chem.MolToSmiles(Chem.MolFromMolBlock(requests.get('https://www.genome.jp/entry/-f+m+C11396').text))

'C=C(C)[C@@H]1CC[C@@H](C)[C@H](O)C1'

In [47]:
from zeep import Client

In [48]:
import hashlib

wsdl = "https://www.brenda-enzymes.org/soap/brenda_zeep.wsdl"
password = hashlib.sha256("bioretro".encode("utf-8")).hexdigest()
client = Client(wsdl)
parameters = ( "itail@mit.edu",password, "ecNumber*1.1", "sequence*", "noOfAminoAcids*", "firstAccessionCode*", "source*", "id*", "organism*")
              
#               "ecNumber*1.1.1.1","organism*Homo sapiens","kmValue*",
#               "kmValueMaximum*","substrate*","commentary*","ligandStructureId*","literature*" )
#               "22834")
# resultString = client.service.getReferenceById(*parameters)
resultString = client.service.getSequence(*parameters)#.getKmValue(*parameters)
print (resultString)

[{
    'firstAccessionCode': 'Q96LJ7',
    'noOfAminoAcids': '313',
    'sequence': 'MAAPMNGQVCVVTGASRGIGRGIALQLCKAGATVYITGRHLDTLRVVAQEAQSLGGQCVPVVCDSSQESEVRSLFEQVDREQQGRLDVLVNNAYAGVQTILNTRNKAFWETPASMWDDINNVGLRGHYFCSVYGARLMVPAGQGLIVVISSPGSLQYMFNVPYGVGKAACDKLAADCAHELRRHGVSCVSLWPGIVQTELLKEHMAKEEVLQDPVLKQFKSAFSSAETTELSGKCVVALATDPNILSLSGKVLPSCDLARRYGLRDVDGRPVQDYLSLSSVLSHVSGLGWLASYLPSFLRVPKWIIALYTSKF',
    'source': 'Swiss-Prot',
    'organism': 'Homo sapiens',
    'id': '32828216',
    'ecNumber': '1.1'
}, {
    'firstAccessionCode': 'A0A0H2ZSR3',
    'noOfAminoAcids': '292',
    'sequence': 'MTTNAAHTTEQLKLGYIGLGNMGAPMATRMTEWPGGVTVYDIRIEAMTPLAEAGATLADSVADVAAADLIHITVLDDAQVREVVGELAGHAKPGTVIAIHSTISDTTAVELARDLKARDIHIVDAPVSGGAAAAARGELATMVGADREVYERIKPAFKHWAAVVIHAGEPGAGTRMKLARNMLTFTSYAAACEAMKLAEAAGLDLQALGRVVRHTDALTGGPGAIMVRDNMKDLEPDNFLYQPFLHTRGLGEKDLSLALALGEAVSVDLPLARLAYEGLAAGLGVPHKEKEA',
    'source': 'TrEMBL',
    'organism': 'Mycobacterium avium (strain 104)',
    'id': '32830385',
    'ecNumb

In [52]:
parameters = ("itail@mit.edu",password,"Zn2+")
resultString = client.service.getLigands(*parameters)
resultString

ValidationError: Missing element organism (getLigands.organism)

In [54]:
import json
with open('brenda/brenda_2023_1.json', 'r') as f:
  brenda_flatfile = json.load(f)

  0%|          | 22/19272 [27:45<404:53:57, 75.72s/it]


In [61]:
brenda_flatfile['data']['1.1.1.1'].keys()

dict_keys(['id', 'name', 'systematic_name', 'synonyms', 'localization', 'tissue', 'cofactor', 'activating_compound', 'inhibitor', 'metals_ions', 'posttranslational_modification', 'subunits', 'engineering', 'cloned', 'organic_solvent_stability', 'expression', 'general_information', 'application', 'reaction_type', 'crystallization', 'purification', 'renaturation', 'general_stability', 'oxygen_stability', 'storage_stability', 'generic_reaction', 'natural_reaction', 'reaction', 'turnover_number', 'km_value', 'ph_optimum', 'ph_range', 'ph_stability', 'specific_activity', 'temperature_optimum', 'temperature_range', 'temperature_stability', 'molecular_weight', 'isoelectric_point', 'ki_value', 'ic50', 'kcat_km', 'organisms', 'references', 'proteins'])

In [106]:
brenda_reactions_text = {}

for br_id in tqdm(bkms['Reaction_ID_BRENDA']):
  if not pd.isnull(br_id):
    b_res = requests.get('https://www.brenda-enzymes.org/structure.php?show=reaction&id={}&type=I&displayType'.format(br_id.replace('BR',''))).text
    brenda_reactions_text[br_id] = b_res


soup = BeautifulSoup(b_res.text, 'html.parser')

  0%|          | 31/42539 [00:39<14:55:01,  1.26s/it]


KeyboardInterrupt: 

In [71]:
for td in soup.find_all('td'):

Reaction catalyzed by [ribulose-bisphosphate carboxylase]-lysine N-methyltransferase (2.1.1.127) 




3

+


=
3

+



3 S-adenosyl-L-methionine
+

 [ribulose-1,5-bisphosphate carboxylase]-L-lysine
=

3 S-adenosyl-L-homocysteine
+

 [ribulose-1,5-bisphosphate carboxylase]-N6,N6,N6-trimethyl-L-lysine


In [103]:
reaction_string = ''
for td in soup.find_all('td'):
  if len(td.text) == 1 or td.text.isnumeric():
    reaction_string += "[" + td.text + "]"
  else:
    a  = td.a
    if a:
      x = re.findall('(?<=brenda_ligand_id\=)[0-9]+', a.get('href'))
      if len(x):
        reaction_string += x[0]

In [104]:
reaction_string

'[3]24[+]169108[=][3]36[+]15333024[+]169108[=]36[+]153330'

In [105]:
soup.find_all('td')

[<td><b>Reaction catalyzed by [ribulose-bisphosphate carboxylase]-lysine N-methyltransferase (2.1.1.127) </b></td>,
 <td align="right">
 <a href="javascript:Sequence('2.1.1.127')"><img alt="show all sequences for 2.1.1.127" border="0" src="/images/nbr_Seq_30.png" title="show all sequences for 2.1.1.127"/></a>
 <a alt="print this document" class="print" href="#print" title="print this document">
 <img border="0" src="/images/print.gif"/></a></td>,
 <td class="larger" valign="middle">3</td>,
 <td valign="middle"><a alt="Go to ligand summary page" href="/ligand.php?brenda_ligand_id=24" target="_blank" title="Go to ligand summary page"><center><img border="0" src="/Mol/Mol.image.php?ID=24&amp;size=small"/></center></a></td>,
 <td class="larger" valign="middle">+</td>,
 <td class="larger" valign="middle"></td>,
 <td valign="middle"><a alt="Go to ligand summary page" href="/ligand.php?brenda_ligand_id=169108" target="_blank" title="Go to ligand summary page"><center><img border="0" src="/Mol